In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/dataset/Quora Question-Pair/Quora Question-pair.zip',compression='zip')

In [ ]:
data['is_duplicate'].value_counts()

0    255027
1    149263
Name: is_duplicate, dtype: int64

In [ ]:
new_data=pd.DataFrame(data[:60000])

In [ ]:
new_data['question1'][0]

'What is the step by step guide to invest in share market in india?'

In [ ]:
new_data['is_duplicate'].value_counts()

0    37600
1    22400
Name: is_duplicate, dtype: int64

In [ ]:
j=0
list1=[]
for i in new_data['is_duplicate']:
  if i==0:
    list1.append(j)
  j+=1
  

In [ ]:
j=0
list3=[]
for i in new_data['is_duplicate']:
  if i==1:
    list3.append(j)
  j+=1

In [ ]:
len(list3)

22400

In [ ]:
len(list1)

37600

In [ ]:
list2=list1[:22400]

In [ ]:
len(list2)

22400

In [ ]:
data1=new_data.iloc[list2]
data2=new_data.iloc[list3]

In [ ]:
nnew_data=pd.concat([data1,data2])

In [ ]:
nnew_data.reset_index(inplace=True)

In [ ]:
from sklearn.utils import shuffle
final_data = shuffle(nnew_data)
final_data.reset_index(inplace=True)

In [ ]:
final_data.columns

Index(['level_0', 'index', 'id', 'qid1', 'qid2', 'question1', 'question2',
       'is_duplicate'],
      dtype='object')

In [ ]:
final_data.drop(columns=['id','qid1','qid2'],inplace=True)
final_data.drop(columns=['level_0','index'],inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
max_len=256
max_words=10000

In [ ]:
import re

In [ ]:
def preprocess(w):
  w=w.lower()
  w=re.sub(r"([!,^#@])",r" ",w)
  w=re.sub(r'[" "]+'," ",w)
  w=re.sub(r"[^a-zA-Z?.!,?$&]+"," ",w)
  w=w.strip()
 # w=w
  return w

In [ ]:
question1=list(map(preprocess,final_data['question1']))

In [ ]:
question1

In [ ]:
question2=list(map(preprocess,final_data['question2']))

In [ ]:
train_question1_data=question1[:35000]
test_question1_data=question1[35000:]
train_question2_data=question2[:35000]
test_question2_data=question2[35000:]
train_duplicate_data=final_data['is_duplicate'][:35000]
test_duplicate_data=final_data['is_duplicate'][35000:]

In [ ]:
! pip install transformers
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
inputs=tokenizer(train_question1_data,train_question2_data,return_tensors='tf',padding=True,truncation=True,max_length=64)
inputs1=tokenizer(test_question1_data,test_question2_data,return_tensors='tf',padding=True,truncation=True,max_length=64)

In [ ]:
tokenizer.get_vocab

<bound method BertTokenizer.get_vocab of PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})>

In [ ]:
train_data1=inputs['input_ids']
test_data1=inputs1['input_ids']

In [ ]:
import xgboost
xgb_reg = xgboost.XGBClassifier(n_estimators=100, learning_rate=0.05,max_depth=1, random_state=42 )

In [ ]:
from scipy import sparse

In [ ]:
xgb_reg.fit(train_data1,train_duplicate_data)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=1,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
xgb_reg.score(test_data1,test_duplicate_data)

0.5964285714285714

In [ ]:

from sklearn.ensemble import VotingClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(random_state=42)

In [ ]:
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
]

In [ ]:
estimators = [random_forest_clf, extra_trees_clf, svm_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(train_data1,train_duplicate_data)

Training the RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
Training the ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                 

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
[estimator.score(test_data1,test_duplicate_data) for estimator in estimators]

[0.6598979591836734, 0.6508163265306123, 0.4925510204081633]

In [ ]:
from sklearn.ensemble import VotingClassifier
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
]

In [ ]:
voting_clf = VotingClassifier(named_estimators)

In [ ]:
voting_clf.fit(train_data1,train_duplicate_data)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_lea

In [ ]:
voting_clf.score(test_data1,test_duplicate_data)

0.6571428571428571

In [ ]:
from tensorflow import keras

In [ ]:
inputs11 = tf.keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = tf.keras.layers.Embedding(30523, 128)(inputs11)
# Add 2 bidirectional LSTMs
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128))(x)
x=tf.keras.layers.Dense(128, activation="relu")(x)
# Add a classifier
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs11, outputs)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_4 (Embedding)      (None, None, 128)         3906944   
_________________________________________________________________
bidirectional_6 (Bidirection (None, None, 512)         788480    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 256)               656384    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 5,384,833
Trainable params: 5,384,833
Non-trainable params: 0
_________________________________________________

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(train_data1,train_duplicate_data, batch_size=32, epochs=2,validation_data=(test_data1,test_duplicate_data))

Epoch 1/2
1094/1094 [==============================] - 75s 60ms/step - loss: 0.6151 - accuracy: 0.6547 - val_loss: 0.5738 - val_accuracy: 0.7000
Epoch 2/2
1094/1094 [==============================] - 64s 59ms/step - loss: 0.4895 - accuracy: 0.7699 - val_loss: 0.5691 - val_accuracy: 0.7086


In [ ]:
inputs111= tf.keras.Input(shape=(None,), dtype="int32")
y = tf.keras.layers.Embedding(30523, 128)(inputs111)
y = tf.keras.layers.LSTM(256, return_sequences=True)(y)
y = tf.keras.layers.LSTM(128, return_sequences=True)(y)
y = tf.keras.layers.LSTM(64,recurrent_dropout=0.2)(y)
y=tf.keras.layers.Dense(128, activation="relu",kernel_regularizer='l2')(y)
y=tf.keras.layers.Dense(64, activation="relu",kernel_regularizer='l2')(y)
outputs1 = tf.keras.layers.Dense(1, activation="sigmoid")(y)
model1 = tf.keras.Model(inputs111, outputs1)
model1.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_8 (Embedding)      (None, None, 128)         3906944   
_________________________________________________________________
lstm_15 (LSTM)               (None, None, 256)         394240    
_________________________________________________________________
lstm_16 (LSTM)               (None, None, 128)         197120    
_________________________________________________________________
lstm_17 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_15 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_16 (Dense)             (None, 64)                8256

In [ ]:
model1.compile(optimizer="adagrad", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model1.fit(train_data1,train_duplicate_data, batch_size=64, epochs=3,validation_data=(test_data1,test_duplicate_data))

Epoch 1/3
547/547 [==============================] - 133s 237ms/step - loss: 2.3865 - accuracy: 0.5132 - val_loss: 2.3014 - val_accuracy: 0.4936
Epoch 2/3
547/547 [==============================] - 126s 231ms/step - loss: 2.2746 - accuracy: 0.5463 - val_loss: 2.1966 - val_accuracy: 0.5740
Epoch 3/3
547/547 [==============================] - 124s 227ms/step - loss: 2.1720 - accuracy: 0.5524 - val_loss: 2.1002 - val_accuracy: 0.5759
